# Import Libraries

In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
import torch
!pip install optuna
import optuna
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, accuracy_score,
                            f1_score, precision_score, recall_score)
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

# Prepare Dataset

In [ ]:
df = pd.read_csv("/content/Dataset-MisinformationData - Sheet1.csv")
label_map = {'real': 1, 'fake': 0}
df['label'] = df['label'].map(label_map)

# Split into train+val (80%) and test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Preprocessing

In [ ]:
nltk.download('punkt_tab')
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'&\w+;', '', text)
    text = re.sub(r'[^\w\s#@]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

train_df['clean_tweet'] = train_df['tweet'].apply(preprocess_text)
test_df['clean_tweet'] = test_df['tweet'].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Get Representations from BERT Models

In [ ]:
models = {
    "bert-base-uncased": "bert-base-uncased",
    "covid-twitter-bert": "digitalepidemiologylab/covid-twitter-bert",
    "twhin-bert-base": "twitter/twhin-bert-base",
    "socbert": "sarkerlab/SocBERT-base"
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_embeddings(model_name, texts):
    tokenizer = AutoTokenizer.from_pretrained(models[model_name])
    model = AutoModel.from_pretrained(models[model_name]).to(device)
    model.eval()

    embeddings = []
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt",
                             truncation=True, padding=True,
                             max_length=128).to(device)
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[0, 0].cpu().numpy()
            embeddings.append(cls_embedding)
    return np.array(embeddings)

# Get embeddings for all models
embeddings = {}
for model_name in models:
    print(f"Getting embeddings for {model_name}...")
    train_emb = get_embeddings(model_name, train_df['clean_tweet'].tolist())
    test_emb = get_embeddings(model_name, test_df['clean_tweet'].tolist())
    embeddings[model_name] = {
        'train': train_emb,
        'test': test_emb
    }

Getting embeddings for bert-base-uncased...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/572M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at sarkerlab/SocBERT-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/572M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at sarkerlab/SocBERT-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Tuning with Optuna

In [ ]:
def objective(trial, X, y):
    params = {
        'C': trial.suggest_float('C', 1e-3, 1e3, log=True),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear']),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }

    cv = StratifiedKFold(n_splits=3)
    scores = []

    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        clf = LogisticRegression(**params, max_iter=1000)
        clf.fit(X_train, y_train)
        preds = clf.predict(X_val)
        scores.append(f1_score(y_val, preds, average='macro'))

    return np.mean(scores)

best_params = {}
for model_name in models:
    print(f"\nTuning {model_name}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial,
                                         embeddings[model_name]['train'],
                                         train_df['label'].values),
                 n_trials=1)

    best_params[model_name] = study.best_params
    print(f"Best params for {model_name}: {study.best_params}")


[I 2025-04-07 04:43:49,664] A new study created in memory with name: no-name-5dfa3b3e-ace6-48e2-848e-8b665feb113d



Tuning bert-base-uncased...


[I 2025-04-07 04:44:00,069] Trial 0 finished with value: 0.8768968931148485 and parameters: {'C': 5.503198476061749, 'solver': 'lbfgs', 'class_weight': None}. Best is trial 0 with value: 0.8768968931148485.
[I 2025-04-07 04:44:00,073] A new study created in memory with name: no-name-5bbff105-1b8f-4dfc-9120-9d7f718a5937


Best params for bert-base-uncased: {'C': 5.503198476061749, 'solver': 'lbfgs', 'class_weight': None}

Tuning covid-twitter-bert...


[I 2025-04-07 04:44:24,703] Trial 0 finished with value: 0.8978967526498925 and parameters: {'C': 650.9831255872026, 'solver': 'liblinear', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.8978967526498925.
[I 2025-04-07 04:44:24,707] A new study created in memory with name: no-name-1346456a-3747-48e3-90e9-dfcb9ecfb04f


Best params for covid-twitter-bert: {'C': 650.9831255872026, 'solver': 'liblinear', 'class_weight': 'balanced'}

Tuning twhin-bert-base...


[I 2025-04-07 04:44:28,659] Trial 0 finished with value: 0.9104947867864216 and parameters: {'C': 3.644827083986285, 'solver': 'lbfgs', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.9104947867864216.
[I 2025-04-07 04:44:28,664] A new study created in memory with name: no-name-b6b894ea-0877-47bf-8931-bde7c34517f8


Best params for twhin-bert-base: {'C': 3.644827083986285, 'solver': 'lbfgs', 'class_weight': 'balanced'}

Tuning socbert...


[I 2025-04-07 04:44:53,687] Trial 0 finished with value: 0.8555091108908993 and parameters: {'C': 47.70440541726406, 'solver': 'liblinear', 'class_weight': None}. Best is trial 0 with value: 0.8555091108908993.


Best params for socbert: {'C': 47.70440541726406, 'solver': 'liblinear', 'class_weight': None}


# Final Evaluation

In [ ]:
def evaluate_model(y_true, y_pred):
    metrics = {
        'confusion_matrix': confusion_matrix(y_true, y_pred),
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro')
    }
    return metrics

results = {}
for model_name in models:
    # Train final classifier with best params
    clf = LogisticRegression(**best_params[model_name], max_iter=1000)
    clf.fit(embeddings[model_name]['train'], train_df['label'])

    # Predict on test set
    test_preds = clf.predict(embeddings[model_name]['test'])

    # Evaluate
    results[model_name] = evaluate_model(test_df['label'].values, test_preds)

    # Print results
    print(f"\nResults for {model_name}:")
    print(f"Accuracy: {results[model_name]['accuracy']:.4f}")
    print(f"F1 Macro: {results[model_name]['f1_macro']:.4f}")
    print(f"Confusion Matrix:\n{results[model_name]['confusion_matrix']}")


Results for bert-base-uncased:
Accuracy: 0.8848
F1 Macro: 0.8846
Confusion Matrix:
[[822 107]
 [117 898]]

Results for covid-twitter-bert:
Accuracy: 0.8956
F1 Macro: 0.8953
Confusion Matrix:
[[822 107]
 [ 96 919]]

Results for twhin-bert-base:
Accuracy: 0.9110
F1 Macro: 0.9109
Confusion Matrix:
[[852  77]
 [ 96 919]]

Results for socbert:
Accuracy: 0.8951
F1 Macro: 0.8948
Confusion Matrix:
[[818 111]
 [ 93 922]]
